sources are based on: https://github.com/AllenInstitute/MicronsBinder/tree/master/notebooks/mm3_intro 

# setting up workspace

In [1]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
import scipy.spatial as sci_spatial
from scipy.spatial import distance_matrix
from tqdm import tqdm
import csv
import pickle #how to use pickle: https://www.datacamp.com/tutorial/pickle-python-tutorial 
import utils
from nglui import statebuilder
import plotly.figure_factory as ff
import networkx as nx
from itertools import chain, combinations
from scipy.cluster.hierarchy import dendrogram
import random
import sklearn
from collections import defaultdict

client = CAVEclient()

In [2]:
# uncomment the following line below to get new token if one has not previously done so; comment out if one has already done 
# client.auth.get_new_token()

# uncomment the following line below to get new token if one has not previously done so; comment out if one has already done 
# client.auth.save_token(token="55d33f46f502c5c22535abf93c68cdb0")

# double checking the token number 
# auth = client.auth
# print(f"My current token is: {auth.token}")

In [3]:
#load up the dataset through query # no query for minnie35: https://github.com/seung-lab/CAVEclient/issues/49 
client = CAVEclient('minnie65_public_v117') #minnie65_public_v117
# client2 = CAVEclient('minnie35_public_v0 ')

In [4]:
#view the tables we can query from the materialization engine
client.materialize.get_tables()

['nucleus_detection_v0',
 'synapses_pni_2',
 'nucleus_neuron_svm',
 'proofreading_status_public_release',
 'func_unit_em_match_release',
 'allen_soma_ei_class_model_v1',
 'allen_visp_column_soma_coarse_types_v1']

working to connect the pre-post synaptic graph. source: https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/SynapseAndAnnotationQuery.ipynb 

In [5]:
# this shows you the basic information about this datastack within CAVE
# client.info.get_datastack_info()

1. Get the "exteneded neurons" from 'proofreading_status_public_release' (refer to the resource code for loading the file. (also add lines to double check and make sure that pt-root-id indeed equals to the valid-id) 

2.1 After getting all the "extended neurons" IDs we then go back to the method of using these IDs to find the synaptic IDs for each extended neuron ID and set up the dictionary. The dictionary will have the folloiwng structure: Key: synaptic ID (id) Vals: a tuple (pre-syn-neuron id aka pre_pt_root_id, post-syn-neuron id aka post_pt_root_id)

2.2 transofrm the voxel coordinates into the xyz coordinates by element-wise-multiplying voxel pos and (4, 4, 40) to get um in x-y-z coordinates 

3. feed the coordinates info into the kd-tree using fcn from scipy. (source: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) 
4. For each synapse, query the kd-tree with a 10 um ball centered on that synapse, and for any neighboring synapse it finds, check to see if they share the same post segmentation ID.
5. (current state) pick a given point, get its closest (10 um) neighbors and then view those points in the Micronexploer/neuroglancer for visual confirmation/verification.

** will be quite interesting to see number of clusters on a given dendrites.

** And we are also interested in finding the number above that is required to achieve making the spike sequences meaningful when relaying the informaiton. 

1. Get the "exteneded neurons" from 'proofreading_status_public_release' (refer to the resource code for loading the file. (also add lines to double check and make sure that pt-root-id indeed equals to the valid-id)

** definition of "extended": 'extended' indicates that the cell is both clean and all tips have been followed as far as a proofreader was able to.

In [6]:
# Load all the proofreaded neurons (with different proofreadbility)
prf_df=client.materialize.query_table('nucleus_detection_v0')

In [7]:
len(prf_df)

144120

In [8]:
prf_df.head()

,id,valid,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,730537,t,32.307937,0,0,"[381312, 273984, 19993]","[nan, nan, nan]","[nan, nan, nan]"
1,373879,t,229.045043,96218056992431305,864691135538371698,"[228816, 239776, 19593]","[nan, nan, nan]","[nan, nan, nan]"
2,601340,t,426.138010,0,0,"[340000, 279152, 20946]","[nan, nan, nan]","[nan, nan, nan]"
3,201858,t,93.753836,84955554103121097,864691135373893678,"[146848, 213600, 26267]","[nan, nan, nan]","[nan, nan, nan]"
4,600774,t,135.189791,0,0,"[339120, 276112, 19442]","[nan, nan, nan]","[nan, nan, nan]"


In [9]:
all_pt_rood_id_ser = pd.Series(prf_df.loc[:,"pt_root_id"])
valid_pt_rood_ids_set = set()
for i in range(len(all_pt_rood_id_ser)):
    if all_pt_rood_id_ser[i] != 0:
        valid_pt_rood_ids_set.add(all_pt_rood_id_ser[i])
    
print(len(valid_pt_rood_ids_set))

110505


In [10]:
valid_pt_rood_ids_lst = list(valid_pt_rood_ids_set)

In [11]:
neuron_type_df_coverall_notallright=client.materialize.query_table('allen_soma_ei_class_model_v1')

In [12]:
len(neuron_type_df_coverall_notallright)

69957

In [13]:
ids_of_coverall_notallright_ser = pd.Series(neuron_type_df_coverall_notallright.loc[:,"pt_root_id"])
types_of_coverall_notallright_ser = pd.Series(neuron_type_df_coverall_notallright.loc[:,"cell_type"])
ids_of_coverall_notallright_lst = list(ids_of_coverall_notallright_ser)
types_of_coverall_notallright_lst = list(types_of_coverall_notallright_ser)

In [14]:
# # run one time and then save the result 
# # valid_ids
# ext_neuron_valid_ids_lst = []
# with tqdm(total= len(valid_pt_rood_ids_lst)) as pbar:
#     for elem in valid_pt_rood_ids_lst:
#         pbar.update(1)
#         if elem in ids_of_coverall_notallright_lst:
#             elem_idx = ids_of_coverall_notallright_lst.index(elem)
#             if types_of_coverall_notallright_ser[elem_idx] == 'excitatory':
#                 ext_neuron_valid_ids_lst.append(elem)

# utils.save_obj_with_name(ext_neuron_valid_ids_lst, 'unproof_ext_neuron_valid_ids_lst')

In [15]:
valid_ids = utils.load_obj_from_filename('unproof_ext_neuron_valid_ids_lst')
print(len(valid_ids))

56565


In [16]:
# setting up str_num_2_presypneuron and presypneuron_2_str_num to simplify the representation of pre-syp-neuron-id
# using string number (e.g.: "20") to correspond to the long-digit id of the pre-syp-neuron 
num_rep_2_presypneuron = {}
presypneuron_2_num_rep = {}
for i, the_id in enumerate(valid_ids):
    num_rep_2_presypneuron[i] = the_id
    presypneuron_2_num_rep[the_id] = i

In [17]:
#500 Server Error reported but should try again
# neuron_type_df_notcoverall_allright=client.materialize.query_table('allen_visp_column_soma_coarse_types_v1')

# working on sheezneat

2.1 After getting all the "extended neurons" IDs (in this case it will be the valid_ids variable that we have declared earlier on) we then go back to the method of using these IDs to find the synaptic IDs for each extended neuron ID and set up the dictionary. The dictionary will have the folloiwng structure: Key: synaptic ID (id) Vals: a tuple (pre-syn-neuron id aka pre_pt_root_id, post-syn-neuron id aka post_pt_root_id)

2.2 In addition, we also transofrm the voxel coordinates into the xyz coordinates by element-wise-multiplying voxel pos and (4, 4, 40) to get um in x-y-z coordinates

In [22]:
#create syp info and save them (only do this once, then are commented out)
syp_dict, syp_voxel_pos, syp_pos_trackingI’m  = utils.creating_syp_information(valid_ids, len(valid_ids), client)
utils.save_syp_information(syp_dict, syp_voxel_pos, syp_pos_tracking)

 22%|███████▊                           | 12571/56565 [7:13:11<25:15:59,  2.07s/it]


HTTPError: 502 Server Error: Bad Gateway for url: https://minnie.microns-daf.com/materialize/api/v2/datastack/minnie65_public_v117/version/117/table/synapses_pni_2/query?return_pyarrow=True&split_positions=True content:b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [ ]:
#loading premade dataset  
syp_dict_file_name = 'syp_dict'
syp_voxel_pos_file_name = 'syp_voxel_pos'
syp_xyz_pos_file_name = 'syp_xyz_pos'
syp_pos_tracking_file_name = 'syp_pos_tracking'
syp_dict, syp_voxel_pos, syp_xyz_pos, syp_pos_tracking = utils.read_in_syp_information(syp_dict_file_name, syp_voxel_pos_file_name, syp_xyz_pos_file_name, syp_pos_tracking_file_name)

In [ ]:
len(syp_voxel_pos)

In [ ]:
len(syp_xyz_pos)

In [ ]:
# side track to look at the data structure in syp_dict
list_of_syp_dict_keys = list(syp_dict.keys())
syp_dict_num_elems_per_key_dict = {}
for key in list_of_syp_dict_keys:
    elements_of_key = syp_dict[key]
    number_of_elems_of_key = len(elements_of_key)
    if number_of_elems_of_key in syp_dict_num_elems_per_key_dict:
        syp_dict_num_elems_per_key_dict[number_of_elems_of_key] += 1
    else:
        syp_dict_num_elems_per_key_dict[number_of_elems_of_key] = 0
        syp_dict_num_elems_per_key_dict[number_of_elems_of_key] += 1
print(syp_dict_num_elems_per_key_dict)

3. feed the coordinates info into the kd-tree using fcn from scipy. (source: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) 

In [ ]:
syp_pos_kd_tree = sci_spatial.KDTree(syp_xyz_pos, 3)
num_rows_of_syp_positions, num_cols_of_syp_positions = np.shape(syp_xyz_pos)
radius = 1e4

4. For each synapse, query the kd-tree with a 10 um (aka 10,000 nm) ball centered on that synapse, and for any neighboring synapse it finds, check to see if they share the same post segmentation ID. (the end goal is to have a set of tuples for which each tuple represent a candidate sequence of pre-syp-neurons (that are verified) that may be on the same dendrite

5. constructing a dict for which key is the spatially ordered tuple and the value of the key is the number of occurance. 

To get the spatially ordered tuple:

First, we construct a distance matrix (symmatic) for the elements within the tuples. Example: for a set of points say "A, B, C, D". The row will be ABCD and the column will be ABCD. Each entry of the matrix is the distance between a pair of points. (how to create a distance matrix in python: https://stackoverflow.com/questions/29481485/creating-a-distance-matrix)

Second, find the largest value in the matrix and select one of the two points associate with that largest value to be the relative origin. 

Third, we then go to the row associated with the selected point to get all the values. And then we can figure out the spatial ordering of all the rest of the points based on the values. A smaller value corresponds to a point closer to the selected origin and vice versa. 

In [ ]:
num_rows_of_syp_positions, num_cols_of_syp_positions = np.shape(syp_xyz_pos)
print(num_rows_of_syp_positions)

In [ ]:
len(syp_pos_tracking)

In [ ]:
# #this part of the code takes care of the potential of multiple counting same sequence of pre-syp-neurons ]
# #that have the same post-ysp-neurons connections and address the issue of end points of cand seq swap. 
# syp_pos_kd_tree = sci_spatial.KDTree(syp_xyz_pos, 3)
# num_rows_of_syp_positions, num_cols_of_syp_positions = np.shape(syp_xyz_pos)
# radius = 1e4

# seqs_with_post_neuron_lst = {}
# syp_ids_of_seqs_with_post_neuron_lst = {}

# with tqdm(total= num_rows_of_syp_positions) as pbar:
#     for i in range(num_rows_of_syp_positions):
#         pbar.update(1)
#         da_pt_and_its_neighbors_pos_lst = []
#         da_pt_pos = syp_xyz_pos[i]
#         resulting_neighbors_idxes = syp_pos_kd_tree.query_ball_point(da_pt_pos, radius, p=2.0, eps=0, workers=3, return_sorted=None, return_length=False)
#         da_pt_syp_ID = syp_pos_tracking[i]
#         neighbors_IDs_list = []
#         for neighbor_pos_idx in resulting_neighbors_idxes:
#             neighbor_ID = syp_pos_tracking[neighbor_pos_idx]
#             neighbors_IDs_list.append(neighbor_ID)
#         syps_with_same_post_syp_ID_lst = []
#         neur_with_same_post_syp_ID_lst = []
#         da_pt_post_syp_id = syp_dict[da_pt_syp_ID][0][1]
#         for neighbor_id in neighbors_IDs_list:
#             neighbor_pre_syp_id, neighbor_post_syp_id = syp_dict[neighbor_id][0]
#             if neighbor_post_syp_id == da_pt_post_syp_id:
#                 syps_with_same_post_syp_ID_lst.append(neighbor_id)
#                 neur_with_same_post_syp_ID_lst.append(neighbor_pre_syp_id)
#                 da_pt_and_its_neighbors_pos_lst.append(syp_xyz_pos[syp_pos_tracking.index(neighbor_id)])
#         ### cast syps_with_post_syp_ID_lst to np array 
#         syps_with_same_post_syp_ID_lst = np.array(syps_with_same_post_syp_ID_lst)
#         neur_with_same_post_syp_ID_lst = np.array(neur_with_same_post_syp_ID_lst)
        
#         ### sorted based on spatial ordering 
#         da_pt_dist_matrix = distance_matrix(da_pt_and_its_neighbors_pos_lst, da_pt_and_its_neighbors_pos_lst)
#         da_pt_dist_mat_max_vals = np.amax(da_pt_dist_matrix)
#         da_pt_maxval_loc_in_dist_mat = np.where(da_pt_dist_matrix == np.amax(da_pt_dist_matrix))
#         da_pt_anchor_pt_idx = da_pt_maxval_loc_in_dist_mat[0][0] #problem with naming variable 
#         ordered_seq_idxs = np.argsort(da_pt_dist_matrix[da_pt_anchor_pt_idx]) #
#         ordered_syn_ids = tuple(syps_with_same_post_syp_ID_lst[ordered_seq_idxs]) #make syps_... an array instead of list
#         ordered_neur_ids = tuple(neur_with_same_post_syp_ID_lst[ordered_seq_idxs])
        
        
        
# #         da_pt_anchor_pt = syps_with_same_post_syp_ID_lst[da_pt_anchor_pt_idx]
# #         da_pt_anchor_row_array = da_pt_dist_matrix[0] #### da_pt_anchor_pot is not being used here
# #         da_pt_anchor_row_list = da_pt_anchor_row_array.tolist()
# #         da_pt_anchor_row_list_with_index = []

# #         for i, elem in enumerate(da_pt_anchor_row_list):
# #             da_pt_anchor_row_list_with_index.append((elem, i))
# #         sorted_da_pt_anchor_row_list_with_index = sorted(da_pt_anchor_row_list_with_index, key=lambda x: x[0])

# #         da_pt_neighbor_spatial_ordered_idx = []
# #         for elem in sorted_da_pt_anchor_row_list_with_index:
# #             da_pt_neighbor_spatial_ordered_idx.append(elem[1])

# #         da_pt_neighbor_pre_syp_neuron_ordered_IDs_lst = []
# #         for i in da_pt_neighbor_spatial_ordered_idx:
# #             syp_id = syps_with_same_post_syp_ID_lst[i]
# #             pre_syp_neuron, post_syp_neuron = syp_dict[syp_id][0]
# #             num_rep_neighbor_pre_syp_id = presypneuron_2_num_rep[pre_syp_neuron]
# #             if not num_rep_neighbor_pre_syp_id in da_pt_neighbor_pre_syp_neuron_ordered_IDs_lst:
# #                 da_pt_neighbor_pre_syp_neuron_ordered_IDs_lst.append(num_rep_neighbor_pre_syp_id)
# #         da_pt_neighbor_pre_syp_neuron_ordered_IDs_lst_tup = tuple(da_pt_neighbor_pre_syp_neuron_ordered_IDs_lst)

#         if not ordered_neur_ids in seqs_with_post_neuron_lst:
#             seqs_with_post_neuron_lst[ordered_neur_ids] = [da_pt_post_syp_id]
#             syp_ids_of_seqs_with_post_neuron_lst[ordered_neur_ids] = [ordered_syn_ids]
#         else:
#             if not da_pt_post_syp_id in seqs_with_post_neuron_lst[ordered_neur_ids]:
#                 seqs_with_post_neuron_lst[ordered_neur_ids].append(da_pt_post_syp_id)
#                 syp_ids_of_seqs_with_post_neuron_lst[ordered_neur_ids].append(ordered_syn_ids)

In [ ]:
# #only do it once then commented out 
# #for 5um radius save the seqs_with_post_neuron_lst and syp_ids_of_seqs_with_post_neuron_lst
# seqs_with_post_neuron_lst_5um = seqs_with_post_neuron_lst
# syp_ids_of_seqs_with_post_neuron_lst_5um = syp_ids_of_seqs_with_post_neuron_lst 
# utils.save_obj_with_name(seqs_with_post_neuron_lst_5um, 'seqs_with_post_neuron_lst_5um')
# utils.save_obj_with_name(syp_ids_of_seqs_with_post_neuron_lst_5um, 'syp_ids_of_seqs_with_post_neuron_lst_5um')

In [ ]:
# # only do it once then commented out 
# #for 10um radius save the seqs_with_post_neuron_lst and syp_ids_of_seqs_with_post_neuron_lst
# seqs_with_post_neuron_lst_10um = seqs_with_post_neuron_lst
# syp_ids_of_seqs_with_post_neuron_lst_10um = syp_ids_of_seqs_with_post_neuron_lst 
# utils.save_obj_with_name(seqs_with_post_neuron_lst_10um, 'seqs_with_post_neuron_lst_10um')
# utils.save_obj_with_name(syp_ids_of_seqs_with_post_neuron_lst_10um, 'syp_ids_of_seqs_with_post_neuron_lst_10um')

In [ ]:
# loading the saved results 
seqs_with_post_neuron_lst = utils.load_obj_from_filename('seqs_with_post_neuron_lst_10um')
syp_ids_of_seqs_with_post_neuron_lst = utils.load_obj_from_filename('syp_ids_of_seqs_with_post_neuron_lst_10um')

In [ ]:
seq_keys = filter(lambda key: len(key)==4, seqs_with_post_neuron_lst)
for key in seq_keys:
    print([k%10000 for k in key], len(seqs_with_post_neuron_lst[key]))

In [ ]:
occurences = [len(seqs_with_post_neuron_lst[key]) for key in seqs_with_post_neuron_lst if len(key)==3]
occ_array = np.array(occurences)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print('using radius = 5um for the kd tree')
legends = []
for i in range(3, 7):
    occurences = [len(seqs_with_post_neuron_lst[key]) for key in seqs_with_post_neuron_lst if len(key)==i]
    occ_array = np.array(occurences)
    print('ttl number of seqs with len' + str(i) + ' = ' + str(len(occ_array)))
    plt.plot(np.sort(occ_array), np.linspace(0, 1, len(occ_array), endpoint=False))
    plt.ylim([0.5, 1])
    plt.xscale("log")
    legends.append('len' + str(i))
plt.legend(legends)
plt.title("cdf of number of occurance on log scale")
plt.show()

In [ ]:
print(len(occ_array))
plt.plot(np.sort(occ_array), np.linspace(0, 1, len(occ_array), endpoint=False))
plt.ylim([0.98, 1])
plt.xscale("log")

In [ ]:
occurences4 = [len(seqs_with_post_neuron_lst[key]) for key in seqs_with_post_neuron_lst if len(key)==4]
occ_array4 = np.array(occurences4)
print(len(occ_array4))
plt.plot(np.sort(occ_array4), np.linspace(0, 1, len(occ_array4), endpoint=False))
plt.ylim([0.98, 1])
plt.xscale("log")

In [ ]:
occurences5 = [len(seqs_with_post_neuron_lst[key]) for key in seqs_with_post_neuron_lst if len(key)==5]
occ_array5 = np.array(occurences5)
print(len(occ_array5))
plt.plot(np.sort(occ_array5), np.linspace(0, 1, len(occ_array5), endpoint=False))
plt.ylim([0.98, 1])
plt.xscale("log")

In [ ]:
a_lst = utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(3, 0, seqs_with_post_neuron_lst)

In [ ]:
len(a_lst)

In [ ]:
a_lst

In [ ]:
len(utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(4, 0, seqs_with_post_neuron_lst))

In [ ]:
len(utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(5, 0, seqs_with_post_neuron_lst))

In [ ]:
len_3_seq_morethan_1_time_w_post_neuron_lst = utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(5, 0, seqs_with_post_neuron_lst)

In [ ]:
len_3_seq_morethan_1_time_w_syp_ids_lst = utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(3, 0, syp_ids_of_seqs_with_post_neuron_lst)

In [ ]:
# url_collections = utils.neuroglancer_render(client, statebuilder, len_3_seq_morethan_1_time_w_post_neuron_lst, len_3_seq_morethan_1_time_w_syp_ids_lst)

In [ ]:
# url_collections_pd = pd.DataFrame([{'neurons': key, 'url':value} for key, value in url_collections.items()])
# url_collections_pd.to_csv('len_3_seq_all_uniqe_url_collections.csv')

In [ ]:
ns = np.round(np.logspace(np.log10(60), np.log10(480), 8, base=10)).astype(int)
cts = np.zeros((3,8,20))
for k,p in enumerate([3, 4, 5]):
    a_lst = utils.get_seq_with_length_n_of_unique_keys_for_occur_more_than_m_times_w_type_lst(p, 0, seqs_with_post_neuron_lst)
    for j, n in enumerate(ns):
        for i in range(20):
            sampled_neurons = random.sample(valid_ids, n)
            ct = 0 
            for elem in a_lst:
                ct += np.all([e in sampled_neurons for e in elem])
            cts[k,j,i] = ct


In [ ]:
cts[0].mean(axis=1)
# cts[1].mean(axis=1)
# cts[2].mean(axis=1)

In [ ]:
colors = ['tab:blue', 'tab:orange', 'tab:green']
for k,p in enumerate([3, 4, 5]):
    means = cts[k].mean(axis=1)
    stds = cts[k].std(axis=1)
    means_lower = np.clip(means - stds, 0, None)
    means_upper = means + stds
    plt.scatter(ns, means, color = colors[k])
    plt.fill_between(ns, means_lower, means_upper, alpha = 0.1)
plt.xscale('log')
plt.yscale('log')


In [ ]:
import sklearn